In [10]:
import numpy as np
import pandas as pd

import matplotlib.dates as mdates
import statsmodels.stats.power as smp
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, KFold
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from itertools import permutations
from scipy.stats import ttest_ind

from statsmodels.tsa.seasonal import STL

import causalpy as cp

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Data Preperation

In [11]:
# Load in claims data by dma, city, dma from 2024-01-01 onwards
claims = pd.read_csv('claims_by_dma.csv')
claims.head()

,date,dma_name,city,state,new_persons,returning_persons,claims
0,2024-08-04,Birmingham (Ann and Tusc),Cullman,AL,2,37,46
1,2024-08-05,"Washington, DC (Hagrstwn)",Silver Spring,MD,25,88,140
2,2024-08-09,Paducah-Cape Girard-Harsbg,Jackson,MO,2,27,34
3,2024-08-13,Orlando-Daytona Bch-Melbrn,Lady Lake,FL,4,48,58
4,2024-08-02,Omaha,Council Bluffs,IA,13,76,104


In [38]:
# Remove Puerto Rico and Virgin Islands
claims = claims.copy()[~claims['state'].isin(['PR', 'VI'])].reset_index().drop('index', axis = 1)

# Add/Modify columns
claims['city_state'] = claims['city'] + ', ' + claims['state']
claims['date'] = pd.to_datetime(claims['date'])

# Group by dma
claims_dma = claims[['date', 'dma_name', 'new_persons', 'returning_persons', 'claims']].groupby(['date', 'dma_name']).sum().reset_index()
claims_dma = claims_dma.fillna(0)
claims_dma.head()

,date,dma_name,new_persons,returning_persons,claims
0,2024-01-01,Abilene-Sweetwater,16,134,170
1,2024-01-01,"Albany, GA",23,153,215
2,2024-01-01,Albany-Schenectady-Troy,36,193,248
3,2024-01-01,Albuquerque-Santa Fe,74,343,466
4,2024-01-01,"Alexandria, LA",8,66,82


In [39]:
# Create new df with statistics
dma_grouped_df = claims_dma.groupby('dma_name').agg(
    count_date=('date', 'count'),
    avg_new_persons=('new_persons', 'mean'),
    sd_new_persons=('new_persons', 'std'),
    avg_returning_persons=('returning_persons', 'mean'),
    sd_returning_persons=('returning_persons', 'std'), 
    avg_claims = ('claims', 'mean'),
    sd_claims = ('claims', 'std'), 
).reset_index()

dma_grouped_df.head(5)

,dma_name,count_date,avg_new_persons,sd_new_persons,avg_returning_persons,sd_returning_persons,avg_claims,sd_claims
0,Abilene-Sweetwater,276,40.090580,14.185362,313.068841,112.370810,411.913043,148.203986
1,"Albany, GA",276,39.391304,13.779265,289.670290,88.281389,386.804348,119.274922
2,Albany-Schenectady-Troy,276,92.351449,30.027315,504.195652,142.781606,663.112319,190.933701
3,Albuquerque-Santa Fe,276,176.304348,54.383635,910.405797,262.055945,1238.760870,363.760902
4,"Alexandria, LA",276,19.090580,7.546880,143.144928,43.639773,184.188406,57.395667


In [40]:
# Load in mapping csv
yt_test_mapping = pd.read_csv('dma_mapping.csv')

# Merge to dma claims
claims_dma = claims_dma.merge(yt_test_mapping[['dma_name', 'youtube_segments']], on='dma_name', how='left')
claims_dma.head()

,date,dma_name,new_persons,returning_persons,claims,youtube_segments
0,2024-01-01,Abilene-Sweetwater,16,134,170,Treatment
1,2024-01-01,"Albany, GA",23,153,215,Control
2,2024-01-01,Albany-Schenectady-Troy,36,193,248,Control
3,2024-01-01,Albuquerque-Santa Fe,74,343,466,Treatment
4,2024-01-01,"Alexandria, LA",8,66,82,Control


In [41]:
treatment_response = claims_dma[claims_dma['youtube_segments'] == 'Treatment']
treatment_response_daily = treatment_response[['date', 'new_persons', 'returning_persons', 'claims']].groupby('date').sum()

treatment_response_daily.head()

,new_persons,returning_persons,claims
date,,,
2024-01-01,4933,33979,43930
2024-01-02,18699,119445,159397
2024-01-03,19950,113063,153227
2024-01-04,20366,110155,150620
2024-01-05,20396,105041,144305


In [42]:
control_response = claims_dma[claims_dma['youtube_segments'] == 'Control']
control_response_daily = control_response[['date', 'new_persons', 'returning_persons', 'claims']].groupby('date').sum()

control_response_daily.head()

,new_persons,returning_persons,claims
date,,,
2024-01-01,4962,34843,45388
2024-01-02,19260,125142,167687
2024-01-03,20241,119568,162165
2024-01-04,20624,115819,158103
2024-01-05,20766,110529,151827


### Estimated Effect of Spend on Response Variable

In [43]:
dma_spend_yt = pd.read_csv('Campaign performance (7).csv', skiprows = 2)

dma_mapping = {
    'Abilene-Sweetwater TX': 'Abilene-Sweetwater',
    'Albany GA': 'Albany, GA',
    'Albany-Schenectady-Troy NY': 'Albany-Schenectady-Troy',
    'Albuquerque-Santa Fe NM': 'Albuquerque-Santa Fe',
    'Alexandria LA': 'Alexandria, LA',
    'Alpena MI': 'Alpena',
    'Amarillo TX': 'Amarillo',
    'Anchorage AK': 'Anchorage',
    'Atlanta GA': 'Atlanta',
    'Augusta GA': 'Augusta-Aiken',
    'Austin TX': 'Austin',
    'Bakersfield CA': 'Bakersfield',
    'Baltimore MD': 'Baltimore',
    'Bangor ME': 'Bangor',
    'Baton Rouge LA': 'Baton Rouge',
    'Beaumont-Port Arthur TX': 'Beaumont-Port Arthur',
    'Bend OR': 'Bend, OR',
    'Billings MT': 'Billings',
    'Biloxi-Gulfport MS': 'Biloxi-Gulfport',
    'Binghamton NY': 'Binghamton',
    'Birmingham (Ann and Tusc) AL': 'Birmingham (Ann and Tusc)',
    'Bluefield-Beckley-Oak Hill WV': 'Bluefield-Beckley-Oak Hill',
    'Boise ID': 'Boise',
    'Boston MA-Manchester NH': 'Boston (Manchester)',
    'Bowling Green KY': 'Bowling Green',
    'Buffalo NY': 'Buffalo',
    'Burlington VT-Plattsburgh NY': 'Burlington-Plattsburgh',
    'Butte-Bozeman MT': 'Butte-Bozeman',
    'Casper-Riverton WY': 'Casper-Riverton',
    'Cedar Rapids-Waterloo-Iowa City & Dubuque IA': 'Cedar Rapids-Wtrlo-IWC&Dub',
    'Champaign & Springfield-Decatur IL': 'Champaign&Sprngfld-Decatur',
    'Charleston SC': 'Charleston, SC',
    'Charleston-Huntington WV': 'Charleston-Huntington',
    'Charlotte NC': 'Charlotte',
    'Charlottesville VA': 'Charlottesville',
    'Chattanooga TN': 'Chattanooga',
    'Cheyenne WY-Scottsbluff NE': 'Cheyenne-Scottsbluff',
    'Chicago IL': 'Chicago',
    'Chico-Redding CA': 'Chico-Redding',
    'Cincinnati OH': 'Cincinnati',
    'Clarksburg-Weston WV': 'Clarksburg-Weston',
    'Cleveland-Akron (Canton) OH': 'Cleveland-Akron (Canton)',
    'Colorado Springs-Pueblo CO': 'Colorado Springs-Pueblo',
    'Columbia SC': 'Columbia, SC',
    'Columbia-Jefferson City MO': 'Columbia-Jefferson City',
    'Columbus GA': 'Columbus, GA (Opelika, AL)',
    'Columbus OH': 'Columbus, OH',
    'Columbus-Tupelo-West Point MS': 'Columbus-Tupelo-W Pnt-Hstn',
    'Corpus Christi TX': 'Corpus Christi',
    'Dallas-Ft. Worth TX': 'Dallas-Ft. Worth',
    'Davenport IA-Rock Island-Moline IL': 'Davenport-R.Island-Moline',
    'Dayton OH': 'Dayton',
    'Denver CO': 'Denver',
    'Des Moines-Ames IA': 'Des Moines-Ames',
    'Detroit MI': 'Detroit',
    'Dothan AL': 'Dothan',
    'Duluth MN-Superior WI': 'Duluth-Superior',
    'El Paso TX': 'El Paso (Las Cruces)',
    'Elmira (Corning) NY': 'Elmira (Corning)',
    'Erie PA': 'Erie',
    'Eugene OR': 'Eugene',
    'Eureka CA': 'Eureka',
    'Evansville IN': 'Evansville',
    'Fairbanks AK': 'Fairbanks',
    'Fargo-Valley City ND': 'Fargo-Valley City',
    'Flint-Saginaw-Bay City MI': 'Flint-Saginaw-Bay City',
    'Florence-Myrtle Beach SC': 'Myrtle Beach-Florence',
    'Fresno-Visalia CA': 'Fresno-Visalia',
    'Ft. Myers-Naples FL': 'Ft. Myers-Naples',
    'Ft. Smith-Fayetteville-Springdale-Rogers AR': 'Ft. Smith-Fay-Sprngdl-Rgrs',
    'Ft. Wayne IN': 'Ft. Wayne',
    'Gainesville FL': 'Gainesville',
    'Glendive MT': 'Glendive',
    'Grand Junction-Montrose CO': 'Grand Junction-Montrose',
    'Grand Rapids-Kalamazoo-Battle Creek MI': 'Grand Rapids-Kalmzoo-B.Crk',
    'Great Falls MT': 'Great Falls',
    'Green Bay-Appleton WI': 'Green Bay-Appleton',
    'Greensboro-High Point-Winston Salem NC': 'Greensboro-H.Point-W.Salem',
    'Greenville-New Bern-Washington NC': 'Greenville-N.Bern-Washngtn',
    'Greenville-Spartanburg-Asheville-Anderson': 'Greenvll-Spart-Ashevll-And',
    'Greenwood-Greenville MS': 'Greenwood-Greenville',
    'Harlingen-Weslaco-Brownsville-McAllen TX': 'Harlingen-Wslco-Brnsvl-McA',
    'Harrisburg-Lancaster-Lebanon-York PA': 'Harrisburg-Lncstr-Leb-York',
    'Harrisonburg VA': 'Harrisonburg',
    'Hartford & New Haven CT': 'Hartford & New Haven',
    'Hattiesburg-Laurel MS': 'Hattiesburg-Laurel',
    'Helena MT': 'Helena',
    'Honolulu HI': 'Honolulu',
    'Houston TX': 'Houston',
    'Huntsville-Decatur (Florence) AL': 'Huntsville-Decatur (Flor)',
    'Idaho Falls-Pocatello ID': 'Idaho Fals-Pocatllo(Jcksn)',
    'Indianapolis IN': 'Indianapolis',
    'Jackson MS': 'Jackson, MS',
    'Jackson TN': 'Jackson, TN',
    'Jacksonville FL': 'Jacksonville',
    'Johnstown-Altoona-dma College PA': 'Johnstown-Altoona-St Colge',
    'Jonesboro AR': 'Jonesboro',
    'Joplin MO-Pittsburg KS': 'Joplin-Pittsburg',
    'Juneau AK': 'Juneau',
    'Kansas City MO': 'Kansas City',
    'Knoxville TN': 'Knoxville',
    'La Crosse-Eau Claire WI': 'La Crosse-Eau Claire',
    'Lafayette IN': 'Lafayette, IN',
    'Lafayette LA': 'Lafayette, LA',
    'Lake Charles LA': 'Lake Charles',
    'Lansing MI': 'Lansing',
    'Laredo TX': 'Laredo',
    'Las Vegas NV': 'Las Vegas',
    'Lexington KY': 'Lexington',
    'Lima OH': 'Lima',
    'Lincoln & Hastings-Kearney NE': 'Lincoln & Hastings-Krny',
    'Little Rock-Pine Bluff AR': 'Little Rock-Pine Bluff',
    'Los Angeles CA': 'Los Angeles',
    'Louisville KY': 'Louisville',
    'Lubbock TX': 'Lubbock',
    'Macon GA': 'Macon',
    'Madison WI': 'Madison',
    'Mankato MN': 'Mankato',
    'Marquette MI': 'Marquette',
    'Medford-Klamath Falls OR': 'Medford-Klamath Falls',
    'Memphis TN': 'Memphis',
    'Meridian MS': 'Meridian',
    'Miami-Ft. Lauderdale FL': 'Miami-Ft. Lauderdale',
    'Milwaukee WI': 'Milwaukee',
    'Minneapolis-St. Paul MN': 'Minneapolis-St. Paul',
    'Minot-Bismarck-Dickinson(Williston) ND': 'Minot-Bsmrck-Dcknsn(Wlstn)',
    'Missoula MT': 'Missoula',
    'Mobile AL-Pensacola (Ft. Walton Beach) FL': 'Mobile-Pensacola (Ft Walt)',
    'Monroe LA-El Dorado AR': 'Monroe-El Dorado',
    'Monterey-Salinas CA': 'Monterey-Salinas',
    'Montgomery-Selma AL': 'Montgomery-Selma',
    'Nashville TN': 'Nashville',
    'New Orleans LA': 'New Orleans',
    'New York NY': 'New York',
    'Norfolk-Portsmouth-Newport News VA': 'Norfolk-Portsmth-Newpt Nws',
    'North Platte NE': 'North Platte',
    'Odessa-Midland TX': 'Odessa-Midland',
    'Oklahoma City OK': 'Oklahoma City',
    'Omaha NE': 'Omaha',
    'Orlando-Daytona Beach-Melbourne FL': 'Orlando-Daytona Bch-Melbrn',
    'Ottumwa IA-Kirksville MO': 'Ottumwa-Kirksville',
    'Paducah KY-Cape Girardeau MO-Harrisburg-Mount Vernon IL': 'Paducah-Cape Girard-Harsbg',
    'Palm Springs CA': 'Palm Springs',
    'Panama City FL': 'Panama City',
    'Parkersburg WV': 'Parkersburg',
    'Peoria-Bloomington IL': 'Peoria-Bloomington',
    'Philadelphia PA': 'Philadelphia',
    'Phoenix AZ': 'Phoenix (Prescott)',
    'Pittsburgh PA': 'Pittsburgh',
    'Portland OR': 'Portland, OR',
    'Portland-Auburn ME': 'Portland-Auburn',
    'Presque Isle ME': 'Presque Isle',
    'Providence-New Bedford MA': 'Providence-New Bedford',
    'Quincy IL-Hannibal MO-Keokuk IA': 'Quincy-Hannibal-Keokuk',
    'Raleigh-Durham (Fayetteville) NC': 'Raleigh-Durham (Fayetvlle)',
    'Rapid City SD': 'Rapid City',
    'Reno NV': 'Reno',
    'Richmond-Petersburg VA': 'Richmond-Petersburg',
    'Roanoke-Lynchburg VA': 'Roanoke-Lynchburg',
    'Rochester NY': 'Rochester, NY',
    'Rochester-Mason City-Austin IA': 'Rochestr-Mason City-Austin',
    'Rockford IL': 'Rockford',
    'Sacramento-Stockton-Modesto CA': 'Sacramnto-Stkton-Modesto',
    'Salisbury MD': 'Salisbury',
    'Salt Lake City UT': 'Salt Lake City',
    'San Angelo TX': 'San Angelo',
    'San Antonio TX': 'San Antonio',
    'San Diego CA': 'San Diego',
    'San Francisco-Oakland-San Jose CA': 'San Francisco-Oak-San Jose',
    'Santa Barbara-Santa Maria-San Luis Obispo CA': 'SantaBarbra-SanMar-SanLuOb',
    'Savannah GA': 'Savannah',
    'Seattle-Tacoma WA': 'Seattle-Tacoma',
    'Sherman-Ada OK': 'Sherman-Ada',
    'Shreveport LA': 'Shreveport',
    'Sioux City IA': 'Sioux City',
    'Sioux Falls(Mitchell) SD': 'Sioux Falls(Mitchell)',
    'South Bend-Elkhart IN': 'South Bend-Elkhart',
    'Spokane WA': 'Spokane',
    'Springfield MO': 'Springfield, MO',
    'Springfield-Holyoke MA': 'Springfield-Holyoke',
    'St. Joseph MO': 'St. Joseph',
    'St. Louis MO': 'St. Louis',
    'Syracuse NY': 'Syracuse',
    'Tallahassee FL-Thomasville GA': 'Tallahassee-Thomasville',
    'Tampa-St. Petersburg (Sarasota) FL': 'Tampa-St. Pete (Sarasota)',
    'Terre Haute IN': 'Terre Haute',
    'Toledo OH': 'Toledo',
    'Topeka KS': 'Topeka',
    'Traverse City-Cadillac MI': 'Traverse City-Cadillac',
    'Tri-Cities TN-VA': 'Tri-Cities, TN-VA',
    'Tucson (Sierra Vista) AZ': 'Tucson (Sierra Vista)',
    'Tulsa OK': 'Tulsa',
    'Twin Falls ID': 'Twin Falls',
    'Tyler-Longview(Lufkin & Nacogdoches) TX': 'Tyler-Longview(Lfkn&Ncgd)',
    'Utica NY': 'Utica',
    'Victoria TX': 'Victoria',
    'Waco-Temple-Bryan TX': 'Waco-Temple-Bryan',
    'Washington DC (Hagerstown MD)': 'Washington, DC (Hagrstwn)',
    'Watertown NY': 'Watertown',
    'Wausau-Rhinelander WI': 'Wausau-Rhinelander',
    'West Palm Beach-Ft. Pierce FL': 'West Palm Beach-Ft. Pierce',
    'Wheeling WV-Steubenville OH': 'Wheeling-Steubenville',
    'Wichita Falls TX & Lawton OK': 'Wichita Falls & Lawton',
    'Wichita-Hutchinson KS': 'Wichita-Hutchinson Plus',
    'Wilkes Barre-Scranton PA': 'Wilkes Barre-Scranton-Hztn',
    'Wilmington NC': 'Wilmington',
    'Yakima-Pasco-Richland-Kennewick WA': 'Yakima-Pasco-Rchlnd-Knnwck',
    'Youngstown OH': 'Youngstown',
    'Yuma AZ-El Centro CA': 'Yuma-El Centro',
    'Zanesville OH': 'Zanesville'
}

dma_spend_yt['dma_name'] = dma_spend_yt['DMA Region (Matched)'].replace(dma_mapping)

/var/folders/n4/96b1c6px5kz8sdgcmpq0wy7h0000gp/T/ipykernel_99979/515580883.py:1: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  dma_spend_yt = pd.read_csv('Campaign performance (7).csv', skiprows = 2)


In [44]:
# Group by dma and date
dma_spend_yt_grouped = dma_spend_yt[['Day', 'dma_name', 'Cost']].groupby(['dma_name', 'Day']).sum()
dma_spend_yt_grouped = dma_spend_yt_grouped.reset_index()

# Map YT Segments to grouped df
spend_by_segment = dma_spend_yt_grouped.merge(yt_test_mapping[['dma_name', 'youtube_segments']], on='dma_name', how='left')

# Group by segments
spend_by_segment_daily = spend_by_segment[['Day', 'Cost', 'youtube_segments']].groupby(['youtube_segments','Day']).sum()

spend_by_segment_daily.head()

Cost
youtube_segments Day                 
Control          2024-01-01   1043.70
                 2024-01-02   7268.44
                 2024-01-03  21312.74
                 2024-01-04  31174.44
                 2024-01-05   6985.82

In [45]:
control_spend_daily = spend_by_segment_daily.loc['Control']
treatment_spend_daily = spend_by_segment_daily.loc['Treatment']

# Ensure both DataFrames have datetime indices
treatment_spend_daily.index = pd.to_datetime(treatment_spend_daily.index)
treatment_response_daily.index = pd.to_datetime(treatment_response_daily.index)

# Rename the indexes to a common name temporarily for merging
treatment_spend_daily = treatment_spend_daily.rename_axis("date_index")
treatment_response_daily = treatment_response_daily.rename_axis("date_index")

# Merge the DataFrames on the common index
treatment_df = treatment_spend_daily.merge(treatment_response_daily, left_index=True, right_index=True, how="inner")

treatment_df.index = pd.to_datetime(treatment_df.index)

treatment_df['incremental_np_direct'] = treatment_df['Cost']/5.94 ### Assuming a Cost Per NP of $5.94 from Direct
treatment_df['incremental_np_mmm'] = treatment_df['Cost']/31.95 ### Assuming a Cost Per NP of $31.95 from MMM readout

treatment_df['incremental_rp_direct'] = treatment_df['Cost']/4.12 ### Assuming a Cost Per RP of $4.12 from Direct
treatment_df['incremental_rp_mmm'] = treatment_df['Cost']/7.24 ### Assuming a Cost Per RP of $7.24 from MMM readout

treatment_df['incremental_claims_mmm'] = treatment_df['Cost']/3.28 ### Assuming a Cost Per Claim of $3.28 from MMM readout

treatment_df.head()

,Cost,new_persons,returning_persons,claims,incremental_np_direct,incremental_np_mmm,incremental_rp_direct,incremental_rp_mmm,incremental_claims_mmm
date_index,,,,,,,,,
2024-01-01,1085.73,4933,33979,43930,182.782828,33.982160,263.526699,149.962707,331.015244
2024-01-02,6407.44,18699,119445,159397,1078.693603,200.545853,1555.203883,885.005525,1953.487805
2024-01-03,20580.33,19950,113063,153227,3464.702020,644.141784,4995.225728,2842.587017,6274.490854
2024-01-04,31716.71,20366,110155,150620,5339.513468,992.698279,7698.230583,4380.761050,9669.728659
2024-01-05,7249.55,20396,105041,144305,1220.462963,226.902973,1759.599515,1001.319061,2210.228659


In [46]:
marginal_effect_np_direct = treatment_df.incremental_np_direct.mean() / treatment_df.new_persons.mean()
marginal_effect_np_mmm = treatment_df.incremental_np_mmm.mean() / treatment_df.new_persons.mean()

marginal_effect_rp_direct = treatment_df.incremental_rp_direct.mean() / treatment_df.returning_persons.mean()
marginal_effect_rp_mmm = treatment_df.incremental_rp_mmm.mean() / treatment_df.returning_persons.mean()

marginal_effect_cl_mmm = treatment_df.incremental_claims_mmm.mean() / treatment_df.claims.mean()

print('Marginal effect of of $1 spend on NP, Direct Attribution:', marginal_effect_np_direct)
print('Marginal effect of of $1 spend on NP, MMM Attribution:', marginal_effect_np_mmm)
print('Marginal effect of of $1 spend on RP, Direct Attribution:', marginal_effect_rp_direct)
print('Marginal effect of of $1 spend on RP, MMM Attribution:', marginal_effect_rp_mmm)
print('Marginal effect of of $1 spend on Claims, MMM Attribution:', marginal_effect_cl_mmm)

Marginal effect of of $1 spend on NP, Direct Attribution: 0.07676315359751665
Marginal effect of of $1 spend on NP, MMM Attribution: 0.014271459542073517
Marginal effect of of $1 spend on RP, Direct Attribution: 0.01863923263549284
Marginal effect of of $1 spend on RP, MMM Attribution: 0.01060685614063957
Marginal effect of of $1 spend on Claims, MMM Attribution: 0.017619922351886117


### Building Synthetic Control

In [63]:
seed = 34
treatment_time = pd.to_datetime("2024-10-02")

In [48]:
# dma pivots by kpi
dma_pivot_np = claims_dma.pivot(index='date', columns='dma_name', values='new_persons')
dma_pivot_rp = claims_dma.pivot(index='date', columns='dma_name', values='returning_persons')
dma_pivot_cl = claims_dma.pivot(index='date', columns='dma_name', values='claims')

In [49]:
# Lists of dmas by Segment
untreated = yt_test_mapping[yt_test_mapping['youtube_segments'] == 'Control'].dma_name.to_list()
treated_1 = yt_test_mapping[yt_test_mapping['youtube_segments'] == 'Treatment'].dma_name.to_list()

In [50]:
# Aggregate Treatments for each KPI - median
dma_pivot_np['treated_agg'] = dma_pivot_np[treated_1].median(axis = 1)
dma_pivot_rp['treated_agg'] = dma_pivot_rp[treated_1].median(axis = 1)
dma_pivot_cl['treated_agg'] = dma_pivot_cl[treated_1].median(axis = 1)

#### New Persons

In [51]:
filtered_cols = treated_1 + untreated
filtered_cols.append('treated_agg')

In [52]:
# Renaming columns
treated_1_renamed = [f't{i+1}' for i in range(len(treated_1))]
untreated_renamed = [f'u{i+1}' for i in range(len(untreated))]

# Combine the two renamed lists
filtered_cols_renamed = treated_1_renamed + untreated_renamed
filtered_cols_renamed.append('treated_agg')

In [53]:
formula = f"treated_agg ~ 0 + {' + '.join(untreated_renamed)}"
print(formula)

treated_agg ~ 0 + u1 + u2 + u3 + u4 + u5 + u6 + u7 + u8 + u9 + u10 + u11 + u12 + u13 + u14 + u15 + u16 + u17 + u18 + u19 + u20 + u21 + u22 + u23 + u24 + u25 + u26 + u27 + u28 + u29 + u30 + u31 + u32 + u33 + u34 + u35 + u36 + u37 + u38 + u39 + u40 + u41 + u42 + u43 + u44 + u45 + u46 + u47 + u48 + u49 + u50 + u51 + u52 + u53 + u54 + u55 + u56 + u57 + u58 + u59 + u60 + u61 + u62 + u63 + u64 + u65 + u66 + u67 + u68 + u69 + u70 + u71 + u72 + u73 + u74 + u75 + u76 + u77 + u78 + u79 + u80 + u81 + u82 + u83 + u84 + u85 + u86 + u87 + u88 + u89 + u90 + u91 + u92 + u93 + u94 + u95 + u96 + u97 + u98 + u99 + u100 + u101 + u102 + u103 + u104


In [58]:
treatment_1_pivot_np = dma_pivot_np[filtered_cols]
treatment_1_pivot_np.columns = filtered_cols_renamed
treatment_1_pivot_np.head()

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,...,u96,u97,u98,u99,u100,u101,u102,u103,u104,treated_agg
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,16.0,74.0,4.0,36.0,34.0,53.0,9.0,89.0,8.0,20.0,...,1.0,47.0,10.0,89.0,4.0,49.0,46.0,20.0,6.0,26.0
2024-01-02,69.0,232.0,2.0,104.0,119.0,348.0,49.0,333.0,58.0,63.0,...,8.0,154.0,49.0,506.0,20.0,127.0,220.0,75.0,20.0,101.0
2024-01-03,65.0,277.0,10.0,112.0,114.0,423.0,49.0,300.0,38.0,83.0,...,12.0,170.0,56.0,456.0,40.0,108.0,239.0,59.0,28.0,108.0
2024-01-04,66.0,208.0,9.0,97.0,134.0,360.0,55.0,350.0,52.0,91.0,...,10.0,176.0,44.0,441.0,25.0,115.0,240.0,79.0,27.0,104.5
2024-01-05,65.0,276.0,10.0,93.0,120.0,376.0,50.0,363.0,53.0,58.0,...,9.0,186.0,58.0,487.0,30.0,140.0,255.0,91.0,21.0,110.5


In [64]:
treatment_1_pivot_np

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,...,u96,u97,u98,u99,u100,u101,u102,u103,u104,treated_agg
date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,16.0,74.0,4.0,36.0,34.0,53.0,9.0,89.0,8.0,20.0,...,1.0,47.0,10.0,89.0,4.0,49.0,46.0,20.0,6.0,26.0
2024-01-02,69.0,232.0,2.0,104.0,119.0,348.0,49.0,333.0,58.0,63.0,...,8.0,154.0,49.0,506.0,20.0,127.0,220.0,75.0,20.0,101.0
2024-01-03,65.0,277.0,10.0,112.0,114.0,423.0,49.0,300.0,38.0,83.0,...,12.0,170.0,56.0,456.0,40.0,108.0,239.0,59.0,28.0,108.0
2024-01-04,66.0,208.0,9.0,97.0,134.0,360.0,55.0,350.0,52.0,91.0,...,10.0,176.0,44.0,441.0,25.0,115.0,240.0,79.0,27.0,104.5
2024-01-05,65.0,276.0,10.0,93.0,120.0,376.0,50.0,363.0,53.0,58.0,...,9.0,186.0,58.0,487.0,30.0,140.0,255.0,91.0,21.0,110.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-28,31.0,125.0,7.0,47.0,19.0,207.0,31.0,167.0,14.0,44.0,...,5.0,80.0,25.0,237.0,12.0,74.0,99.0,38.0,15.0,52.0
2024-09-29,20.0,88.0,3.0,33.0,16.0,136.0,24.0,124.0,12.0,29.0,...,4.0,78.0,19.0,144.0,8.0,48.0,62.0,34.0,10.0,32.0
2024-09-30,50.0,245.0,4.0,118.0,40.0,373.0,61.0,325.0,33.0,74.0,...,9.0,141.0,31.0,381.0,18.0,116.0,140.0,75.0,24.0,90.5


In [60]:
treatment_1_pivot_np = treatment_1_pivot_np.fillna(0)

In [69]:
pooled = cp.SyntheticControl(
    treatment_1_pivot_np,
    treatment_time,
    formula=formula,
    model=cp.pymc_models.WeightedSumFitter(
        sample_kwargs={
            "target_accept": 0.95,
            "random_seed": seed,
            "cores": 1,        # Add cores=1 to force single-core processing
            "chains": 1        # Add chains=1 to minimize multiprocessing
        }
    ),
)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [beta, sigma]


/opt/anaconda3/envs/google_tests_3/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 127 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks
Sampling: [beta, sigma, y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]


In [70]:
# Apply coefficients to unweighted dma data
coefficients_posterior = pooled.idata.posterior['beta'] 
coefficients_mean = coefficients_posterior.mean(dim=['chain', 'draw'])
coefficients_list = coefficients_mean.values.tolist()

untreated_weighted = dma_pivot_np[untreated] * coefficients_list

In [71]:
control_response = untreated_weighted.sum(axis = 1)
treatment_response = dma_pivot_np.treated_agg

In [72]:
control_response

date
2024-01-01     26.215979
2024-01-02     96.064763
2024-01-03    101.568479
2024-01-04    102.583358
2024-01-05    102.534700
                 ...    
2024-09-28     49.627786
2024-09-29     35.248667
2024-09-30     86.594372
2024-10-01     94.795443
2024-10-02     60.191546
Length: 276, dtype: float64

In [73]:
# Pooled estimated standard deviations
def sample_means(control_response, n):
    control_sample_means = []
    treatment_sample_means = []

    # Cycle through iterations of samples of size N
    for start in range(len(control_response) - n + 1):
        sample_control = control_response.iloc[start:start + n]
        sample_treatment = treatment_response.iloc[start:start + n]
        
        # Calculate the mean of this sample and store it
        control_sample_means.append(sample_control.mean())
        treatment_sample_means.append(sample_treatment.mean())
    
    # Calculate the mean and standard deviation of the sample means
    control_mean_of_means = np.mean(control_sample_means)
    control_std_dev_of_means = np.std(control_sample_means)

    treatment_mean_of_means = np.mean(treatment_sample_means)
    treatment_std_dev_of_means = np.std(treatment_sample_means)

    pooled_std = np.sqrt(((treatment_std_dev_of_means ** 2) + (control_std_dev_of_means ** 2))/2)
    
    
    return control_mean_of_means, treatment_mean_of_means, pooled_std 

n = 30  # For example, 30 days
control_mean_of_means, treatment_mean_of_means, pooled_std = sample_means(control_response, n)

print("Mean of control sample means:", control_mean_of_means)
print("Standard deviation of sample means:", pooled_std)

Mean of control sample means: 69.23612626728323
Standard deviation of sample means: 3.632034790886931


In [74]:
print(marginal_effect_np_mmm)
print(marginal_effect_np_direct)

0.014271459542073517
0.07676315359751665


In [77]:
# Power Analysis - MMM
diff_means_mmm = (treatment_response*marginal_effect_np_mmm).mean()
effect_size = diff_means_mmm / pooled_std
print('Effect Size:', effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='larger')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.2736080032833855
Days Neccassary: 165.85411652308437


In [79]:
# Power Analysis - Direct
diff_means_direct = (treatment_response*marginal_effect_np_direct).mean()
effect_size = diff_means_direct / pooled_std
print('Effect Size:', effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='larger')
print('Days Neccassary:',observations_neccasary)

Effect Size: 1.4716794116000282
Days Neccassary: 6.507555665157363


In [82]:
# Power Analysis - Average
diff_means_avg = (diff_means_direct + diff_means_mmm) / 2
effect_size = diff_means_avg / pooled_std
print('Effect Size:', effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='larger')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.8726437074417069
Days Neccassary: 16.956827434052695


### Returning Persons

In [83]:
# Set target df
treatment_1_pivot_rp = dma_pivot_rp[filtered_cols]
treatment_1_pivot_rp.columns = filtered_cols_renamed

In [85]:
pooled = cp.SyntheticControl(
    treatment_1_pivot_rp,
    treatment_time,
    formula=formula,
    model=cp.pymc_models.WeightedSumFitter(
        sample_kwargs={
            "target_accept": 0.95,
            "random_seed": seed,
            "cores": 1,        # Add cores=1 to force single-core processing
            "chains": 1        # Add chains=1 to minimize multiprocessing
        }
    ),
)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [beta, sigma]


/opt/anaconda3/envs/google_tests_3/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 125 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks
Sampling: [beta, sigma, y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]


In [86]:
# Apply coefficients to unweighted dma data
coefficients_posterior = pooled.idata.posterior['beta'] 
coefficients_mean = coefficients_posterior.mean(dim=['chain', 'draw'])
coefficients_list = coefficients_mean.values.tolist()

untreated_weighted = dma_pivot_rp[untreated] * coefficients_list

In [87]:
control_response = untreated_weighted.sum(axis = 1)
treatment_response = dma_pivot_rp.treated_agg

In [88]:
# Pooled estimated standard deviations
def sample_means(control_response, n):
    control_sample_means = []
    treatment_sample_means = []

    # Cycle through iterations of samples of size N
    for start in range(len(control_response) - n + 1):
        sample_control = control_response.iloc[start:start + n]
        sample_treatment = treatment_response.iloc[start:start + n]
        
        # Calculate the mean of this sample and store it
        control_sample_means.append(sample_control.mean())
        treatment_sample_means.append(sample_treatment.mean())
    
    # Calculate the mean and standard deviation of the sample means
    control_mean_of_means = np.mean(control_sample_means)
    control_std_dev_of_means = np.std(control_sample_means)

    treatment_mean_of_means = np.mean(treatment_sample_means)
    treatment_std_dev_of_means = np.std(treatment_sample_means)

    pooled_std = np.sqrt(((treatment_std_dev_of_means ** 2) + (control_std_dev_of_means ** 2))/2)
    
    
    return control_mean_of_means, treatment_mean_of_means, pooled_std 

n = 30  # For example, 30 days
control_mean_of_means, treatment_mean_of_means, pooled_std = sample_means(control_response, n)

print("Mean of control sample means:", control_mean_of_means)
print("Standard deviation of sample means:", pooled_std)

Mean of control sample means: 430.8594186524475
Standard deviation of sample means: 9.064203989098552


In [89]:
print(marginal_effect_rp_direct)
print(marginal_effect_rp_mmm)

0.01863923263549284
0.01060685614063957


In [90]:
# Power Analysis - MMM
diff_means_mmm = (treatment_response*marginal_effect_rp_mmm).mean()
effect_size = diff_means_mmm / pooled_std
print('Effect Size:',effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=-minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='smaller')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.5077953541604124
Days Neccassary: 48.64430848119634


In [91]:
# Power Analysis - Direct
diff_means_direct = (treatment_response*marginal_effect_rp_direct).mean()
effect_size = diff_means_direct / pooled_std
print('Effect Size:',effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=-minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='smaller')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.8923394087673268
Days Neccassary: 16.24990756185917


In [92]:
# Power Analysis - Average
diff_means_avg = (diff_means_mmm + diff_means_direct) / 2
effect_size = diff_means_avg / pooled_std
print('Effect Size:',effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=-minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='smaller')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.7000673814638696
Days Neccassary: 25.93388052704876


### Claims

In [93]:
treatment_1_pivot_cl = dma_pivot_cl[filtered_cols]
treatment_1_pivot_cl.columns = filtered_cols_renamed

In [94]:
pooled = cp.SyntheticControl(
    treatment_1_pivot_cl,
    treatment_time,
    formula=formula,
    model=cp.pymc_models.WeightedSumFitter(
        sample_kwargs={
            "target_accept": 0.95,
            "random_seed": seed,
            "cores": 1,        # Add cores=1 to force single-core processing
            "chains": 1        # Add chains=1 to minimize multiprocessing
        }
    ),
)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [beta, sigma]


/opt/anaconda3/envs/google_tests_3/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 126 seconds.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks
Sampling: [beta, sigma, y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]


In [95]:
# Apply coefficients to unweighted dma data
coefficients_posterior = pooled.idata.posterior['beta'] 
coefficients_mean = coefficients_posterior.mean(dim=['chain', 'draw'])
coefficients_list = coefficients_mean.values.tolist()

untreated_weighted = dma_pivot_rp[untreated] * coefficients_list

In [96]:
control_response = untreated_weighted.sum(axis = 1)
treatment_response = dma_pivot_rp.treated_agg

In [97]:
# Pooled estimated standard deviations
def sample_means(control_response, n):
    control_sample_means = []
    treatment_sample_means = []

    # Cycle through iterations of samples of size N
    for start in range(len(control_response) - n + 1):
        sample_control = control_response.iloc[start:start + n]
        sample_treatment = treatment_response.iloc[start:start + n]
        
        # Calculate the mean of this sample and store it
        control_sample_means.append(sample_control.mean())
        treatment_sample_means.append(sample_treatment.mean())
    
    # Calculate the mean and standard deviation of the sample means
    control_mean_of_means = np.mean(control_sample_means)
    control_std_dev_of_means = np.std(control_sample_means)

    treatment_mean_of_means = np.mean(treatment_sample_means)
    treatment_std_dev_of_means = np.std(treatment_sample_means)

    pooled_std = np.sqrt(((treatment_std_dev_of_means ** 2) + (control_std_dev_of_means ** 2))/2)
    
    
    return control_mean_of_means, treatment_mean_of_means, pooled_std 

n = 30  # For example, 30 days
control_mean_of_means, treatment_mean_of_means, pooled_std = sample_means(control_response, n)

print("Mean of control sample means:", control_mean_of_means)
print("Standard deviation of sample means:", pooled_std)

Mean of control sample means: 429.1534875081307
Standard deviation of sample means: 9.124049475827302


In [98]:
print(marginal_effect_cl_mmm)

0.017619922351886117


In [99]:
# Power Analysis - MMM
diff_means_mmm = (treatment_response*marginal_effect_cl_mmm).mean()
effect_size = diff_means_mmm / pooled_std
print('Effect Size:',effect_size)
minimum_effect_size = effect_size

analysis = smp.TTestIndPower()
observations_neccasary = analysis.solve_power(effect_size=-minimum_effect_size, nobs1 = None, alpha=.05, power=.8, ratio=1.0, alternative='smaller')
print('Days Neccassary:',observations_neccasary)

Effect Size: 0.838007824946748
Days Neccassary: 18.32345934130849
